**Preamble**

**Colloboration Policy**. The student is to *explicitly identify* his/her collaborators in the assignment. If the student did not work with anyone, he/she should indicate `Collaborators=['none']`. If the student obtains a solution through research (e.g., on the web), acknowledge the source, but *write up the solution in HIS/HER OWN WORDS*. There will be a one mark penalty if a student fails to indicate his/her collaborators.

**There will be NO EXCEPTIONS to this grading policy.**

## Transportation Problem

If you need help on using iPython notebooks, click <a href='#help'>here</a>. 

Assignment objectives:

i. Familiarize with the PuLP syntax and use PuLP to solve transportation problems. Click <a href='#transport_help'>here</a> for a working example.

ii. Solve *unbalanced* transportation problems by (a) using PuLP directly, and (b) introducing a dummy city and using PuLP to solve the balanced version. You expect that the answers from both methods are the same.

-----

### Mortarboard Problem

Your company manufactures mortarboards and supplies to the six universities (NUS, NTU, SMU, SUTD, SIT, SUSS) in Singapore. You have have three factories in three locations: Jurong, Woodlands, Changi.

The demand, supply and transportation costs are summarized in the following table.

|         |NUS|NTU|SMU|SUTD|SIT|SUSS|Supply|
|---------|---|---|---|----|---|----|------|
|Jurong   | 14| 2 | 25| 37 | 16| 14 | 40   |
|Woodlands| 22| 21| 29| 37 | 23| 20 | 40   |
|Changi   | 28| 37| 19| 1  | 30| 26 | 40   |
|Demand   | 39| 31| 10| 1  | 5 | 13 |


**(a) (7 marks)** Using PuLP directly, minimize the transportation costs while meeting the demands of all universities.

<span style="color:blue">Remember to display the costs and the number of mortarboards to be transported along each route.</span>


In [30]:
"""
The Mortarboard Problem

Solving the unbalanced version
"""

import pulp

# Supply factory list
Factories = ["Jurong", "Woodlands", "Changi"]

# Supply factory capacity dictionary
Supply = {
    Factories[0] : 40,
    Factories[1] : 40,
    Factories[2] : 40 }

# Demand university list
Universities = ["NUS", "NTU", "SMU", "SUTD", "SIT", "SUSS"]

# Demand university dictionary
Demand = {
    Universities[0] : 39,
    Universities[1] : 31,
    Universities[2] : 10,
    Universities[3] : 1, 
    Universities[4] : 5, 
    Universities[5] : 13 }

# Cost Matrix for each factory<->university path
Costs = { 
    Factories[0] : 
        {
            Universities[0] : 14, 
            Universities[1] : 2, 
            Universities[2] : 25, 
            Universities[3] : 37, 
            Universities[4] : 16,
            Universities[5] : 14
        },
    Factories[1] : 
        {
            Universities[0] : 22, 
            Universities[1] : 21, 
            Universities[2] : 29, 
            Universities[3] : 37, 
            Universities[4] : 23,
            Universities[5] : 20
        },
    Factories[2] : 
        {
            Universities[0] : 28, 
            Universities[1] : 37, 
            Universities[2] : 19, 
            Universities[3] : 1, 
            Universities[4] : 30,
            Universities[5] : 26
        }}

# Problem LP
Prob = pulp.LpProblem("The Mortarboard Problem", pulp.LpMinimize)

# All possible route tuples
Routes = [(mF, mU) for mF in Factories for mU in Universities]

# Reference variable dictionary
RouteVars = pulp.LpVariable.dicts("Route", (Factories, Universities), lowBound=0)

# Add the objective function
Prob += pulp.lpSum([RouteVars[mF][mU] * Costs[mF][mU] for (mF, mU) in Routes]), "Sum of Costs"


# Add the supply constaints
for mF in Factories:
    Prob += pulp.lpSum([RouteVars[mF][mU] for mU in Universities]) <= Supply[mF], "Total supply from %s"%mF

# Add the demand constraints 
for mU in Universities:
    Prob += pulp.lpSum([RouteVars[mF][mU] for mF in Factories]) >= Demand[mU], "Total demand from %s"%mU

print(Prob)

Prob.solve()

print("model status: ", pulp.LpStatus[Prob.status])

print("Minimum Cost: {}".format(pulp.value(Prob.objective)))

for (mF, mU) in Routes:
    print("Amount through {}: {} with total costs {} at {} per unit".format
          (RouteVars[mF][mU], RouteVars[mF][mU].varValue, 
           RouteVars[mF][mU].varValue * Costs[mF][mU], Costs[mF][mU]))

The Mortarboard Problem:
MINIMIZE
37*Route_Changi_NTU + 28*Route_Changi_NUS + 30*Route_Changi_SIT + 19*Route_Changi_SMU + 26*Route_Changi_SUSS + 1*Route_Changi_SUTD + 2*Route_Jurong_NTU + 14*Route_Jurong_NUS + 16*Route_Jurong_SIT + 25*Route_Jurong_SMU + 14*Route_Jurong_SUSS + 37*Route_Jurong_SUTD + 21*Route_Woodlands_NTU + 22*Route_Woodlands_NUS + 23*Route_Woodlands_SIT + 29*Route_Woodlands_SMU + 20*Route_Woodlands_SUSS + 37*Route_Woodlands_SUTD + 0
SUBJECT TO
Total_supply_from_Jurong: Route_Jurong_NTU + Route_Jurong_NUS
 + Route_Jurong_SIT + Route_Jurong_SMU + Route_Jurong_SUSS + Route_Jurong_SUTD
 <= 40

Total_supply_from_Woodlands: Route_Woodlands_NTU + Route_Woodlands_NUS
 + Route_Woodlands_SIT + Route_Woodlands_SMU + Route_Woodlands_SUSS
 + Route_Woodlands_SUTD <= 40

Total_supply_from_Changi: Route_Changi_NTU + Route_Changi_NUS
 + Route_Changi_SIT + Route_Changi_SMU + Route_Changi_SUSS + Route_Changi_SUTD
 <= 40

Total_demand_from_NUS: Route_Changi_NUS + Route_Jurong_NUS
 + Route

---
Notice that the total supply (120) exceeds total demand (99). In other words, this is an *unbalanced* transportation problem.

To create a *balanced* instance, we introduce a "dummy" university, whose demand is the excess supply (21=120-99). The costs from each factory to the dummy university is set to zero. In other words, the modified demand, supply and transportation costs are summarized in the following table.

|         |NUS|NTU|SMU|SUTD|SIT|SUSS|Dummy|Supply|
|---------|---|---|---|----|---|----|-----|------|
|Jurong   | 14| 2 | 25| 37 | 16| 14 | 0   | 40   |
|Woodlands| 22| 21| 29| 37 | 23| 20 | 0   | 40   |
|Changi   | 28| 37| 19| 1  | 30| 26 | 0   | 40   |
|Demand   | 39| 31| 10| 1  | 5 | 13 | 21  |


**(b) (7 marks)** Using PuLP to minimize the transportation costs for the modified problem.

<span style="color:blue">Remember to display the costs and the number of mortarboards to be transported along each route.


In [31]:
"""
The Mortarboard Problem(BALANCED)

Solving the balanced version with Dummy University
"""

import pulp

# Supply factory list
Factories = ["Jurong", "Woodlands", "Changi"]

# Supply factory capability dictionary
Supply = {
    Factories[0] : 40,
    Factories[1] : 40,
    Factories[2] : 40 }

# Demand university list with DUMMY
Universities = ["NUS", "NTU", "SMU", "SUTD", "SIT", "SUSS", "DUMMY"] 

# Demand university dictionary with DUMMY
Demand = {
    Universities[0] : 39,
    Universities[1] : 31,
    Universities[2] : 10,
    Universities[3] : 1, 
    Universities[4] : 5, 
    Universities[5] : 13,
    Universities[6] : 21} # THE DUMMY

# Cost Matrix for each factory<->university path, with all factories to DUMMY's cost set to ZERO
Costs = { 
    Factories[0] : 
        {
            Universities[0] : 14, 
            Universities[1] : 2, 
            Universities[2] : 25, 
            Universities[3] : 37, 
            Universities[4] : 16,
            Universities[5] : 14,
            Universities[6] : 0 # THE DUMMY
        },
    Factories[1] : 
        {
            Universities[0] : 22, 
            Universities[1] : 21, 
            Universities[2] : 29, 
            Universities[3] : 37, 
            Universities[4] : 23,
            Universities[5] : 20,
            Universities[6] : 0 # THE DUMMY
        },
    Factories[2] : 
        {
            Universities[0] : 28, 
            Universities[1] : 37, 
            Universities[2] : 19, 
            Universities[3] : 1, 
            Universities[4] : 30,
            Universities[5] : 26,
            Universities[6] : 0 # THE DUMMY
        }}

# Problem LP
Prob = pulp.LpProblem("The Mortarboard Problem(BALANCED)", pulp.LpMinimize)

# All possible route tuples
Routes = [(mF, mU) for mF in Factories for mU in Universities]

# Reference variable dictionary
RouteVars = pulp.LpVariable.dicts("Route", (Factories, Universities), lowBound=0)

# Add the objective function
Prob += pulp.lpSum([RouteVars[mF][mU] * Costs[mF][mU] for (mF, mU) in Routes]), "Sum of Costs"


# Add the supply constaints
for mF in Factories:
    Prob += pulp.lpSum([RouteVars[mF][mU] for mU in Universities]) == Supply[mF], "Total supply from %s"%mF

# Add the demand constraints 
for mU in Universities:
    Prob += pulp.lpSum([RouteVars[mF][mU] for mF in Factories]) == Demand[mU], "Total demand from %s"%mU

print(Prob)

Prob.solve()

print("model status: ", pulp.LpStatus[Prob.status])

print("Minimum Cost: {}".format(pulp.value(Prob.objective)))

for (mF, mU) in Routes:
    print("Amount through {}: {} with total costs {} at {} per unit".format
          (RouteVars[mF][mU], RouteVars[mF][mU].varValue, 
           RouteVars[mF][mU].varValue * Costs[mF][mU], Costs[mF][mU]))

The Mortarboard Problem(BALANCED):
MINIMIZE
37*Route_Changi_NTU + 28*Route_Changi_NUS + 30*Route_Changi_SIT + 19*Route_Changi_SMU + 26*Route_Changi_SUSS + 1*Route_Changi_SUTD + 2*Route_Jurong_NTU + 14*Route_Jurong_NUS + 16*Route_Jurong_SIT + 25*Route_Jurong_SMU + 14*Route_Jurong_SUSS + 37*Route_Jurong_SUTD + 21*Route_Woodlands_NTU + 22*Route_Woodlands_NUS + 23*Route_Woodlands_SIT + 29*Route_Woodlands_SMU + 20*Route_Woodlands_SUSS + 37*Route_Woodlands_SUTD + 0
SUBJECT TO
Total_supply_from_Jurong: Route_Jurong_DUMMY + Route_Jurong_NTU
 + Route_Jurong_NUS + Route_Jurong_SIT + Route_Jurong_SMU + Route_Jurong_SUSS
 + Route_Jurong_SUTD = 40

Total_supply_from_Woodlands: Route_Woodlands_DUMMY + Route_Woodlands_NTU
 + Route_Woodlands_NUS + Route_Woodlands_SIT + Route_Woodlands_SMU
 + Route_Woodlands_SUSS + Route_Woodlands_SUTD = 40

Total_supply_from_Changi: Route_Changi_DUMMY + Route_Changi_NTU
 + Route_Changi_NUS + Route_Changi_SIT + Route_Changi_SMU + Route_Changi_SUSS
 + Route_Changi_SUTD 

**(c) (1 mark)** In the optimal solution, what is the amount of mortarboards left in each factory?

**Answer:**
**There are multiple ways to approach this, easily from Part (b)'s last print statement, we can see that all leftovers are with Changi,**

* Amount through Route_Jurong_DUMMY: **0.0** with total costs 0.0 at 0 per unit

* Amount through Route_Woodlands_DUMMY: **0.0** with total costs 0.0 at 0 per unit

* Amount through Route_Changi_DUMMY: **21.0** with total costs 0.0 at 0 per unit

**Hence,**

* Leftover in Jurong:      **0** 

* Leftover in Woodlands:   **0** 

* Leftover in Changi:      **21**